In [1]:
import matplotlib
from src.repository.surebet_repository import SurebetRepository
from src.genetic.arbitrage import CrossOverEnum, Population
from src.util import ExportadorDeGraficos, PickleSerializer
repository = SurebetRepository()

<h3> Parâmetros para execução do experimento </h3>

In [2]:
n_generations = 200
n_individuals = 20
mutation_rate = 0.01
crossover_rate = 0.95

In [3]:
def paretos_frontier_plot(front, selected_individuals, path):
    plt = matplotlib.pyplot
    plt.figure()
    # plt.xlim(0, 1.05)
    # plt.ylim(0, 1.05)
    plt.title(f'Generation')
    plt.xlabel('Profit')
    plt.ylabel('Dispersation')
    plt.scatter([i.fitness_normalized[0] for i in front], [i.fitness_normalized[1] for i in front])
    plt.scatter([i.fitness_normalized[0] for i in selected_individuals], [i.fitness_normalized[1] for i in selected_individuals], color='green')
    plt.savefig(path)
    plt.close()


def pick_up_individuals(front):   
    front_fitness = list({tuple(i.fitness) for i in front})  # Remove os itens repetidos
    OPTIMIZE_PROFIT = -1
    OPTIMIZE_MIDDLE = (len(front_fitness) // 2)
    OPTIMIZE_DISPERSATION = 0
    
    front_sorted = sorted(front, key=lambda x: x.fitness[0])
    front_fitness = sorted(front_fitness, key=lambda x: x[0])
    
    print(front_fitness)
    
    aux_individuals_selected = front_fitness[OPTIMIZE_PROFIT], front_fitness[OPTIMIZE_MIDDLE], front_fitness[OPTIMIZE_DISPERSATION]  # Seleciona os 3 indivíduos (Aqui só tem os fitness)
    print(f'aux_individuals_selected: {aux_individuals_selected}')
    selected = []
    
    # Método para descobrir qual a é o indivíduo(dentro da fronteira de pareto que possui vários objetos) que possui o fitness escolhido anteriormente
    for i in aux_individuals_selected:
        for individual in front_sorted:
            if list(i) == individual.fitness:
                selected.append(individual)
                break
                
    print(f'selected: {selected}')
    return front_sorted, selected

def normalize(front, high_fitness_evaluated):
    max_profit = high_fitness_evaluated[0]
    max_dispersation = high_fitness_evaluated[1]
    for i in front:
        profit_normalized = i.fitness[0] / max_profit
        dispersation_normalized = i.fitness[1] / max_dispersation
        i.fitness_normalized = [profit_normalized, dispersation_normalized]
    

In [4]:
def export_frontier(export, population: Population):
    generation_plot = [0, population.n_generations - 1, population.best_frontier]
    for generation, front in population.pareto_history_front_normalized:
        x = [i[0] for i in front]
        y = [i[1] for i in front]
        if generation in generation_plot:
            export.plot(x, y, generation)

def export_solution(population: Population, front_sorted, individuals_selected):
    export = ExportadorDeGraficos()
    export.hypervolume_plot(population)
    PickleSerializer.save(population, f'{export.data_hora_atual}/pickle.pkl')
    path = export.data_hora_atual + '/' + export.get_file_name(population.best_frontier, 'png')
    export_frontier(export, population)
    paretos_frontier_plot(front_sorted, individuals_selected, path)
    
    
def plot_frontier(population: Population):
    frontier = population.pareto_history_front[population.best_frontier][1]
    front_sorted, individuals_selected = pick_up_individuals(frontier)
    
    normalize(front_sorted, population.high_fitness_evaluated)
    normalize(individuals_selected, population.high_fitness_evaluated)
    
    export_solution(population, front_sorted, individuals_selected)
    
    return individuals_selected

<h3> Cenários mês de setembro </h3>
<img src="images/surebets_setembro.png" width="75%">
<br> <br>
<img src="images/surebets_02_setembro.png" width="75%">

<h3> Cenário 02/09/2023 </h3>

In [5]:
start_date = '2023-09-02 14:00:00'
end_date = '2023-09-02 14:05:00'
arbitrages = repository.find_all_unique_between(start_date, end_date)
population = Population(n_generations=n_generations, n_individuals=n_individuals, mutation_rate=mutation_rate, crossover_rate=crossover_rate, 
                        budget=1000, crossover_strategy=CrossOverEnum.ONE_POINT_CROSSOVER, arbitrages=arbitrages)

individuals_picked_up = plot_frontier(population)


None
138
[(12.395100000000001, 2.3295940852632238), (13.2867, 1.9868687756988326), (21.5055, 1.5355180332029907), (24.7468, 0.6577047787442195), (28.3019, 0.0)]
aux_individuals_selected: ((28.3019, 0.0), (21.5055, 1.5355180332029907), (12.395100000000001, 2.3295940852632238))
selected: [<src.genetic.arbitrage.Individual object at 0x7fe435daa590>, <src.genetic.arbitrage.Individual object at 0x7fe434b6b850>, <src.genetic.arbitrage.Individual object at 0x7fe4353d7eb0>]
[28.3019, 0.0]: [(1, 1.0), (0, 0.14), (0, 0.41), (0, 0.44), (0, 0.13)]
[21.5055, 1.5355180332029907]: [(1, 0.63), (1, 0.13), (1, 0.11), (1, 0.13), (0, 0.13)]
[12.395100000000001, 2.3295940852632238]: [(1, 0.19), (1, 0.21), (1, 0.17), (1, 0.21), (1, 0.24)]



### Cenário 24/09/2023
